# Building the Question Answering System

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from nltk.tokenize import word_tokenize
import keras

Using TensorFlow backend.


# Preprocessing

#### Hyperparameter Notes
-Word vector size (50,100,200,300)
-how to deal with tokens with no word vectorization (REMOVE, generate random array, spell checker?)

In [2]:
train_df=pd.read_json("C:/Users/Lukas Buteliauskas/Desktop/training_data.json").reset_index(drop=True)
dev_df=pd.read_json("C:/Users/Lukas Buteliauskas/Desktop/validation_data.json").reset_index(drop=True)

## Word Vectorization
To be able to use words, phrases, questions or other natural language constructs in our model we require a to provide our neural network a numerical representation of our words (as these are the elemental NLP 'particles'). The simplest implementation would be to use 'one hot encoding' and define each word as a vector the size of our dictionary (the number of unique words found in our collection of documents, our corpus). However, this approach will most likely be insufficient for the purposes of a question answering system. word2vec and GloVe are 2 popular choices sophisticated options for word embeddings that also capture word similarities. I will not go into the details of either architecture other than to say that we will not be re-training the word vectors due to the insufficient size of the dataset, and we will begin with the GloVe word embeddings due to it's superior performance in most 'downstream' modelling tasks. Having said that, given the simplicity of swapping word vector representations we will also test out performance with word2vec (providing we can do so in a time-efficient manner).

Info and download links for GloVe can be found at: https://nlp.stanford.edu/projects/glove/

### 'Word Vector' Custom Functions

In [3]:
def get_word_vector_dict(url_or_path):
    """Takes a URL or a local path and returns a dictionary of GloVe word vectors where the key is the word and the value is the 
    word vector with the dimension specified in the input file."""
    
    with open(url_or_path, encoding="utf8") as glove_text:
        word_lines=glove_text.readlines()
    word_embeddings=[line.split(" ") for line in word_lines]
    word_vector_dict={element[0]:list(map(float, element[1:])) for element in word_embeddings}
    
    return word_vector_dict


def get_word_vector_df(url_path_or_dict):
    """Takes a URL or path like the previous function, or can take a word vector dictionary and returns a word vector dataframe.
    Rows of the dataframe are the word vectors, columns are the dimensions of the word vector, indices are the words."""
    
    if type(url_path_or_dict) is str:
        with open(url_path_or_dict, encoding="utf8") as glove_text:
            word_lines=glove_text.readlines()
        word_embeddings=[line.split(" ") for line in word_lines]
        word_vector_dict={element[0]:list(map(float, element[1:])) for element in word_embeddings}
        word_vector_df=pd.DataFrame(word_vector_dict).transpose()
    
    else:
        word_vector_df=pd.DataFrame(url_path_or_dict).transpose()
    
    return word_vector_df

### Setting up the Word Vectors
As mentioned above with regards to what model we use for the word vectors, it's important to note that the dimention of the word vectors is a hyperparameter of the Neural Networks to come, so to keep our options open we imported a few different word vectors representations and the custom functions defined above make this a 'one line of code' affair (dictionary or dataframe).


In [4]:
word_vector_50_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.50d.txt")
word_vector_50_df=get_word_vector_df(word_vector_50_dict)
vocab=np.array(word_vector_50_dict.keys()) #400k words as per the documentation.

word_vector_100_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.100d.txt")
word_vector_100_df=get_word_vector_df(word_vector_100_dict)

In [5]:
"""word_vector_200_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.200d.txt")
word_vector_200_df=get_word_vector_df(word_vector_200_dict)

word_vector_300_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.300d.txt")
word_vector_300_df=get_word_vector_df(word_vector_300_dict)"""

'word_vector_200_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.200d.txt")\nword_vector_200_df=get_word_vector_df(word_vector_200_dict)\n\nword_vector_300_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.300d.txt")\nword_vector_300_df=get_word_vector_df(word_vector_300_dict)'

## Ensuring all tokens have word embeddings

### Tokenization, Embedding Custom Functions

In [6]:
def get_unique_tokens(df):
    """Given a dataframe containing contexts, questions and answers, the function returns a list of unique tokens."""
    pieces_of_text=list(df["context"].unique()) + list(df["title"].unique()) + list(df["question"].unique()) 
    pieces_of_text+=list(df["answer_text"].unique())

    non_unique_tokens=[]
    for text in pieces_of_text:
        temp_tokens=word_tokenize(text)
        non_unique_tokens.append(temp_tokens)

    unique_tokens=set()
    for token in non_unique_tokens:
        unique_tokens.update(set(token))
    
    return [token.replace("``", '"').replace("''", '"').lower() for token in list(unique_tokens)]


def split_keep_sep(tokens, sep, only_unique=True):
    """Takes a string or a list of tokens, and splits on 'sep' while keeping sep. Returns a set of unique tokens, or a list
    of tokens after splitting.
    DO NOT PASS IN A STRING OR IT WILL RETURN A BUNCH OF CHARACTERS"""
    split_tokens=[]
    for token in tokens:
        if only_unique==True:
            for sub_token in re.split("("+ sep + ")", token):
                if sub_token not in split_tokens:
                    split_tokens.append(sub_token)
        else: # if we want just a list of all the tokens after seperation
            split_tokens.extend(re.split("("+ sep + ")", token))
                
    return [token for token in split_tokens if token!=""]


def tokenize(string):
    """Takes a string (either context, question or answer) and returns a list of tokens."""
    return [token.replace("``", '"').replace("''", '"').lower() for token in word_tokenize(string)]


def tokenize_further(tokens, seperator_list=["-","–","—","'","£","/",":"]):
    """Further splits the tokens on the seperators, while keeping the seperators as tokens.
    Returns a further tokenized list of tokens."""
    split_tokens=split_keep_sep(tokens, seperator_list[0], only_unique=False)
    for sep in seperator_list[1:]:
        split_tokens=split_keep_sep(split_tokens, sep, only_unique=False)
    
    return [token for token in split_tokens if token!=""] # filters out "" (empty space tokens)

def string_to_embedding(string, word_vector_dict=word_vector_50_dict):
    """Takes a context, question or answer and returns a list of word tokens in their vectorized form."""
    tokens=tokenize_further(tokenize(string))
    embedding=[]  
    
    #Go over each token in the string (after all the preprocessing)
    for token in tokens:
        if token in word_vector_dict.keys(): # if it has a vectorization, add it to the 'embedding' list.
            embedding.extend(word_vector_dict[token])
        else:
            """Apply function to generate random vectorization. CURRENTLY WE ARE IGNORING WORDS THAT DON'T HAVE AN EMBEDDING."""
            
    return embedding

The nltk tokenizer generated around 110,00 unique tokens from our contexts, questions and answers in our dataset. ~31,000 of those tokens did not have pre-trained word vectorizations in the GloVe model. Some of these tokens were numbers, expressed as strings in an unfamiliar format, some of these tokens were misspelled words, some of these tokens were works in other languages, or symbols from other alphibets and so on. 
The ideal case would be to assign to each token in our corpus a 'meaningful' word vectorization, however given the time constraint on this project, we will try to provide meaningful word vectorizations to as many of these tokens as possible through pre-processing steps following the initial word tokenization (using the nltk tokenizer).

In [7]:
unique_tokens=get_unique_tokens(train_df)
no_embeddings=[token for token in unique_tokens if token not in word_vector_50_dict.keys()]
split_unique_tokens=split_keep_sep(unique_tokens, "-")
split_unique_tokens2=split_keep_sep(split_unique_tokens, "–")
split_unique_tokens3=split_keep_sep(split_unique_tokens2, "—")
split_unique_tokens4=split_keep_sep(split_unique_tokens3, "'")
split_unique_tokens5=split_keep_sep(split_unique_tokens4, "£")
split_unique_tokens6=split_keep_sep(split_unique_tokens5, "/")
split_unique_tokens7=split_keep_sep(split_unique_tokens6, ":")

In [8]:
print("Num of initial tokens:", len(unique_tokens), "\nNum of tokens after '-' split:", len(split_unique_tokens))
print("Num of tokens after 2nd split", len(split_unique_tokens2),"\nNum of tokens after 3rd split", len(split_unique_tokens3))
print("Num of tokens after 4th split", len(split_unique_tokens4), "\nNum of tokens after 5th split", len(split_unique_tokens5))
print("Num of tokens after 6th split", len(split_unique_tokens6), "\nNum of tokens after 7th split", len(split_unique_tokens7),"\n")

no_embeddings_2=[token for token in split_unique_tokens if token not in word_vector_50_dict.keys()]
no_embeddings_3=[token for token in split_unique_tokens2 if token not in word_vector_50_dict.keys()]
no_embeddings_4=[token for token in split_unique_tokens3 if token not in word_vector_50_dict.keys()]
no_embeddings_5=[token for token in split_unique_tokens4 if token not in word_vector_50_dict.keys()]
no_embeddings_6=[token for token in split_unique_tokens5 if token not in word_vector_50_dict.keys()]
no_embeddings_7=[token for token in split_unique_tokens6 if token not in word_vector_50_dict.keys()]
no_embeddings_8=[token for token in split_unique_tokens7 if token not in word_vector_50_dict.keys()]

print("Number of tokens with no embedding in GloVe (with the current tokenization):")
print("with '-' with words:",len(no_embeddings),"\nwith '-' seperate:", len(no_embeddings_2),)
print("with 2 - with words:", len(no_embeddings_3),"\nwith all - with words:", len(no_embeddings_4))
print("with all - and ' with words:", len(no_embeddings_5) ,"\nwith all -, ', £ with words:", len(no_embeddings_6))
print("with all -, ', £, / with words:", len(no_embeddings_7),"\nwith all -, ', £, /, : with words:", len(no_embeddings_8),"\n")

print(no_embeddings_8[0:100])

Num of initial tokens: 119319 
Num of tokens after '-' split: 94470
Num of tokens after 2nd split 92798 
Num of tokens after 3rd split 91669
Num of tokens after 4th split 90768 
Num of tokens after 5th split 90639
Num of tokens after 6th split 89796 
Num of tokens after 7th split 89386 

Number of tokens with no embedding in GloVe (with the current tokenization):
with '-' with words: 31521 
with '-' seperate: 24311
with 2 - with words: 22541 
with all - with words: 21396
with all - and ' with words: 20560 
with all -, ', £ with words: 20395
with all -, ', £, / with words: 19604 
with all -, ', £, /, : with words: 19385 

['ummah.com', 'kumārila', 'neighbourhods', 'makhtesh', 'hernandia', 'infrastructure.', '27,799', 'zŏnglĭ', 'whitsand', 'danishnamah', 'stsc', 'iātros', 'asace', 'chaetae', 'g.m.c', '1,530,000', 'paedobaptism', 'khiuma', 'indvuna', 'catepillars', 'finges', 'blueware', 'hutecheson', 'avicide', 'zakria', 'feminity', 'kilikiti', 'linters', 'malevoncia', 'harjumaa', 'animal

In [9]:
embedding_list=string_to_embedding(train_df.loc[0,["context"]][0])
print(embedding_list[0:150],"\n")
print(word_vector_50_dict["beyoncé"],"\n")
print(word_vector_50_dict["giselle"],"\n")
print(word_vector_50_dict["knowles"],"\n")

[-0.73129, 0.62083, 0.04594, 0.77703, -0.5803, 0.27172, 0.28471, 0.32273, 0.8567, 1.5485, 0.54048, 0.21277, 0.13194, 0.013766, -0.15691, -0.396, -0.6855, -0.39768, 0.8253, -0.31198, -0.18205, 1.246, 0.96381, 0.048889, 0.71645, 0.82746, -1.067, -0.38175, -0.37723, -0.74626, 0.11207, 0.17772, 0.59491, -0.6364, 0.17068, -0.50995, 1.0413, -0.60245, -0.73349, -1.0289, -0.18171, -0.49433, -0.34356, -1.2125, 0.43832, -0.51488, -0.037414, -1.9207, -0.2878, 0.79805, 0.44796, 0.20505, -1.7072, 0.25522, -0.61353, 0.45521, 0.5134, 0.36352, 0.13682, -0.49604, 0.58205, 0.90981, -0.83431, 0.37364, 0.7367, 0.15918, 0.27294, 0.23022, 0.56572, -0.85467, 0.13297, 0.76733, -0.04357, -0.031591, 0.45783, 0.37597, -0.8988, 0.94555, -1.3402, -0.46339, -0.92484, 0.73547, -0.0067005, 0.68015, -0.27115, -0.50333, 0.29133, 0.13423, 0.6459, 0.21893, -0.28878, 0.58685, 0.30512, -1.0662, -0.49959, -0.60076, 0.159, 0.020639, 0.28208, 0.54983, -0.82729, 0.1195, 0.39781, 0.81701, -0.1012, 0.33862, -0.0038192, 0.21697, 

In [10]:
print("yon" in word_vector_50_dict.keys())

True
